In [133]:
import numpy as np
import cv2
import matplotlib.pyplot as pp
import math

In [134]:
# First method, from <Fast Noise Variance Estimation>

In [184]:
sigma = 20

In [185]:
S = np.array(cv2.imread("cameraman.png", cv2.IMREAD_GRAYSCALE))
N = np.array([[1, -2, 1], [-2, 4, -2], [1, -2, 1]])

In [186]:
Sw, Sh = S.shape
Nw, Nh = N.shape

In [187]:
SN = np.random.normal(0, sigma, S.shape)
I = S + SN

In [188]:
def multsum(A1, A2):
    return np.sum(np.sum(np.multiply(A1, A2)))

In [189]:
sum = 0
for r in range(0, Sw-2):
    for c in range(0, Sh-2):
        neighbors = I[r:r+3, c:c+3]
        sum += multsum(neighbors, N)**2.0
        

In [190]:
estimated_sigma = np.sqrt(sum/(36*(Sw-2)*(Sh-2)))
print(sigma, estimated_sigma)

20 21.338614198415144


In [200]:
# Second method, from Fast Method for Noise Level Estimation and Integrated Noise Reduction 
# by Angelo Bosco1, Arcangelo Bruna1,  Giuseppe Messina1, Giuseppe Spampinato1
#
# The crux of the algorithm is, 
# 1. Pixel differences in a homogenous region is mostly due to noise
# 2. Almost no noise signal exceeds 3sigma
# 3. Hence, taking the 68th percentile of the noise signals that are within 3sigma
#    will give us an estimation of the actual sigma

In [201]:
sigma_max = sigma*3

In [202]:
def flatten(neighbors):
    return neighbors.reshape(neighbors.size)

def test(neighbors): # tests the homogenity of the neighborhood
    flat = flatten(neighbors)
    centerV = flat[4]
    omit_center = flat[np.arange(flat.size)!=4]
    d = np.abs(omit_center - centerV)
    return np.all(d < sigma_max), d

In [203]:
ds = []
for r in range(0, Sw-2):
    for c in range(0, Sh-2):
        neighbors = I[r:r+3, c:c+3] # might consider another way of picking the neighbors
        is_homogenous, d = test(neighbors)
        ds.extend(d if is_homogenous else [])
     

In [204]:
v, bins = np.histogram(ds, 1000)
cumv = np.cumsum(v[1:])
b = bins[1:]

In [205]:
estimated_sigma = 0
for i in range(len(cumv)):
    if cumv[i] > cumv[-1]*0.68:
        estimated_sigma = b[i]
        break
print(sigma, estimated_sigma)

20 25.85940980433985
